In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
# Importing and concatenating all the datasets

# List of dataset names
dataset_names = [
    '2002-property-sales-data.csv',
    '2003-property-sales-data.csv',
    '2004-property-sales-data.csv',
    '2005-property-sales-data.csv',
    '2006-property-sales-data.csv',
    '2007-property-sales-data.csv',
    '2008-property-sales-data.csv',
    '2009-property-sales-data.csv',
    '2010-property-sales-data.csv',
    '2011-property-sales-data.csv',
    '2012-property-sales-data.csv',
    '2013-property-sales-data.csv',
    '2014-property-sales-data.csv',
    '2015-property-sales-data.csv',
    '2016-property-sales-data.csv',
    '2017-property-sales-data.csv',
    '2018-property-sales-data.csv',
    '2019-property-sales-data.csv',
    'armslengthsales_2020_valid.csv',
    'armslengthsales_2021_valid.csv',
    'armslengthsales_2022_valid_20230404.csv',
    '2002-2018-property-sales-data.csv'
]

# Initialize an empty dataframe to hold the concatenated data
all_data = pd.DataFrame()

# Loop through each dataset and append to the all_data dataframe
for dataset in dataset_names:
    temp_data = pd.read_csv(dataset)
    all_data = pd.concat([all_data, temp_data])

# Reset index
all_data.reset_index(drop=True, inplace=True)

# Display the first few rows of the concatenated dataframe
all_data.head()

In [ ]:
# Convert the 'Sale_date' column to datetime format
all_data['Sale_date'] = pd.to_datetime(all_data['Sale_date'], errors='coerce')

# Drop rows where 'Sale_date' is NaT (Not a Time)
all_data.dropna(subset=['Sale_date'], inplace=True)

# Set 'Sale_date' as the index
all_data.set_index('Sale_date', inplace=True)

# Sort the dataframe by the index
all_data.sort_index(inplace=True)

# Display the first few rows of the dataframe
all_data.head()

In [ ]:
# Exploratory Data Analysis (EDA)

# Importing necessary libraries for visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Summary statistics of the numerical columns
numerical_summary = all_data.describe()

# Count of missing values in each column
missing_values_count = all_data.isnull().sum()

# Count of unique values in each column
unique_values_count = all_data.nunique()

numerical_summary, missing_values_count, unique_values_count

In [ ]:
# Exploratory Data Analysis (EDA)

# Importing necessary libraries for visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Summary statistics of the numerical columns
summary_stats = all_data.describe()

# Count of missing values in each column
missing_values = all_data.isnull().sum()

# Count of unique values in each column
unique_values = all_data.nunique()

summary_stats, missing_values, unique_values

(             Taxkey      District          Nbhd       Stories    Year_Built  \
 count  6.922900e+04  95445.000000  69229.000000  95318.000000  95405.000000   
 mean   3.744019e+09      8.045220   3841.098875      1.336101   1920.165526   
 std    1.482311e+09      4.181342   1802.142029      0.618919    220.137679   
 min    1.002100e+07      1.000000     40.000000      0.000000      0.000000   
 25%    2.712604e+09      4.000000   2100.000000      1.000000   1923.000000   
 50%    3.591595e+09      9.000000   4330.000000      1.000000   1950.000000   
 75%    5.080176e+09     11.000000   5297.000000      2.000000   1960.000000   
 max    7.169999e+09     15.000000   6982.000000     26.000000   2022.000000   
 
           Nr_of_rms       Fin_sqft         Units         Bdrms         Fbath  \
 count  69229.000000   69229.000000  95447.000000  94177.000000  94792.000000   
 mean       0.682691    2555.306635      1.362746      2.940283      1.329026   
 std        1.641299    9290.150111

In [ ]:
# Cleaning the 'Sale_price' column

# Removing dollar signs and commas
all_data['Sale_price'] = all_data['Sale_price'].str.replace('$', '', regex=False)
all_data['Sale_price'] = all_data['Sale_price'].str.replace(',', '', regex=False)

# Converting to float
all_data['Sale_price'] = all_data['Sale_price'].astype(float)

# Display the first few rows to confirm the changes
all_data.head()

In [ ]:
# Investigating Outliers

# Plotting boxplots for 'Year_Built' and 'Fin_sqft'
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.boxplot(x='Year_Built', data=all_data)
plt.title('Boxplot of Year_Built')

plt.subplot(1, 2, 2)
sns.boxplot(x='Fin_sqft', data=all_data)
plt.title('Boxplot of Fin_sqft')
plt.show()

# Exploring the Distribution of Key Variables

# Plotting histograms for 'Sale_price' and 'Year_Built'
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(all_data['Sale_price'], bins=50, kde=True)
plt.title('Distribution of Sale_price')

plt.subplot(1, 2, 2)
sns.histplot(all_data['Year_Built'], bins=50, kde=True)
plt.title('Distribution of Year_Built')
plt.show()

In [ ]:
# Addressing Issues in 'Year_Built' and 'Fin_sqft'

# Replacing 'Year_Built' values of 0 with the median value of the column
median_year = all_data['Year_Built'].median()
all_data['Year_Built'].replace(0, median_year, inplace=True)

# Removing extreme outliers in 'Fin_sqft' based on IQR
Q1 = all_data['Fin_sqft'].quantile(0.25)
Q3 = all_data['Fin_sqft'].quantile(0.75)
IQR = Q3 - Q1

# Defining bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Removing outliers
all_data = all_data[(all_data['Fin_sqft'] >= lower_bound) & (all_data['Fin_sqft'] <= upper_bound)]

# Display the first few rows to confirm the changes
all_data.head()

In [ ]:
# Displaying the column names to identify the correct column for sale dates
all_data.columns

Index(['PropType', 'Taxkey', 'Address', 'District', 'Nbhd', 'Style', 'Extwall',
       'Stories', 'Year_Built', 'Fin_sqft', 'Units', 'Bdrms', 'Fbath', 'Hbath',
       'Lotsize', 'Sale_price', 'PropertyID', 'taxkey', 'nbhd',
       'FinishedSqft'],
      dtype='object')

In [ ]:
# Displaying the column names of 'all_data' to confirm the presence of 'Sale_date'
all_data.columns

Index(['PropType', 'Taxkey', 'Address', 'District', 'Nbhd', 'Style', 'Extwall',
       'Stories', 'Year_Built', 'Fin_sqft', 'Units', 'Bdrms', 'Fbath', 'Hbath',
       'Lotsize', 'Sale_price', 'PropertyID', 'taxkey', 'nbhd',
       'FinishedSqft'],
      dtype='object')

In [ ]:
# Displaying the first few rows of the 2002-property-sales-data.csv dataset to check for 'Sale_date'
data_2002 = pd.read_csv('2002-property-sales-data.csv')
data_2002.head()

In [ ]:
# Checking for the presence of 'Sale_date' column in other datasets
datasets_with_sale_date = []

for dataset in dataset_names:
    temp_data = pd.read_csv(f'{dataset}')
    if 'Sale_date' in temp_data.columns:
        datasets_with_sale_date.append(dataset)

datasets_with_sale_date

['2002-property-sales-data.csv',
 '2003-property-sales-data.csv',
 '2004-property-sales-data.csv',
 '2005-property-sales-data.csv',
 '2006-property-sales-data.csv',
 '2007-property-sales-data.csv',
 '2008-property-sales-data.csv',
 '2009-property-sales-data.csv',
 '2010-property-sales-data.csv',
 '2011-property-sales-data.csv',
 '2012-property-sales-data.csv',
 '2013-property-sales-data.csv',
 '2014-property-sales-data.csv',
 '2015-property-sales-data.csv',
 '2016-property-sales-data.csv',
 '2017-property-sales-data.csv',
 '2018-property-sales-data.csv',
 '2019-property-sales-data.csv',
 'armslengthsales_2020_valid.csv',
 'armslengthsales_2021_valid.csv',
 'armslengthsales_2022_valid_20230404.csv',
 '2002-2018-property-sales-data.csv']

In [ ]:
# Merging datasets that contain the 'Sale_date' column into 'all_data'
all_data = pd.concat([pd.read_csv(dataset) for dataset in datasets_with_sale_date], ignore_index=True)

# Converting 'Sale_date' to datetime format
all_data['Sale_date'] = pd.to_datetime(all_data['Sale_date'])

# Displaying the first few rows of the merged dataset
all_data.head()

In [ ]:
# Cleaning the 'Sale_price' column to convert it to numerical format
all_data['Sale_price'] = all_data['Sale_price'].str.replace('$', '', regex=False)
all_data['Sale_price'] = all_data['Sale_price'].str.replace(',', '', regex=False)
all_data['Sale_price'] = all_data['Sale_price'].astype(float)

# Displaying the first few rows to confirm the changes
all_data.head()

In [ ]:
# Re-attempting to plot the time series data of Sale Prices after cleaning the 'Sale_price' column
plt.figure(figsize=(15, 6))
sns.lineplot(x='Sale_date', y='Sale_price', data=all_data)
plt.title('Time Series Analysis of Property Sale Prices')
plt.xlabel('Sale Date')
plt.ylabel('Sale Price')
plt.show()

In [ ]:
# Importing the seasonal_decompose function for time series decomposition
from statsmodels.tsa.seasonal import seasonal_decompose

# Resampling the data to monthly frequency and taking the mean of Sale Prices for each month
resampled_data = all_data.resample('M', on='Sale_date').Sale_price.mean()

# Decomposing the time series into trend, seasonality, and noise components
decomposition = seasonal_decompose(resampled_data.dropna())

# Plotting the decomposed time series components
plt.figure(figsize=(15, 10))
plt.subplot(4, 1, 1)
plt.plot(resampled_data, label='Original')
plt.legend(loc='upper left')
plt.title('Time Series Decomposition')

plt.subplot(4, 1, 2)
plt.plot(decomposition.trend, label='Trend')
plt.legend(loc='upper left')

plt.subplot(4, 1, 3)
plt.plot(decomposition.seasonal, label='Seasonal')
plt.legend(loc='upper left')

plt.subplot(4, 1, 4)
plt.plot(decomposition.resid, label='Residual')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# Importing ARIMA model from statsmodels
from statsmodels.tsa.arima.model import ARIMA

# Fitting the ARIMA model on the resampled data (ignoring NaN values)
model = ARIMA(resampled_data.dropna(), order=(5,1,0))
model_fit = model.fit()

# Forecasting the next 12 months
forecast = model_fit.forecast(steps=12)

# Plotting the original data and the forecast
plt.figure(figsize=(15, 6))
plt.plot(resampled_data.index, resampled_data.values, label='Original')
plt.plot(pd.date_range(resampled_data.index[-1], periods=12, freq='M'), forecast, label='Forecast')
plt.title('ARIMA Forecast of Property Sale Prices')
plt.xlabel('Date')
plt.ylabel('Sale Price')
plt.legend()
plt.show()

/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


In [ ]:
# Importing Random Forest Regressor and other necessary libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Preparing the data
resampled_data = resampled_data.dropna().reset_index()
X = np.array(range(len(resampled_data))).reshape(-1, 1)
y = resampled_data['Sale_price'].values

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Creating and fitting the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Making predictions
y_pred = rf_model.predict(X_test)
forecast_rf = rf_model.predict(np.array(range(len(resampled_data), len(resampled_data) + 12)).reshape(-1, 1))

# Evaluating the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Plotting the original data and the Random Forest forecast
plt.figure(figsize=(15, 6))
plt.plot(X, y, label='Original')
plt.plot(np.array(range(len(resampled_data), len(resampled_data) + 12)).reshape(-1, 1), forecast_rf, label='Random Forest Forecast')
plt.title(f'Random Forest Forecast of Property Sale Prices (RMSE: {rmse:.2f})')
plt.xlabel('Time Index')
plt.ylabel('Sale Price')
plt.legend()
plt.show()

In [ ]:
# Importing necessary libraries for data manipulation and time series analysis
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Reading the data from the CSV files
data_2008 = pd.read_csv('2008-property-sales-data.csv')
data_2018 = pd.read_csv('2018-property-sales-data.csv')
data_2022 = pd.read_csv('armslengthsales_2022_valid_20230404.csv')

# Concatenating the data into a single DataFrame
all_data = pd.concat([data_2008, data_2018, data_2022])

# Converting 'Sale_date' to datetime format and 'Sale_price' to float
all_data['Sale_date'] = pd.to_datetime(all_data['Sale_date'])
all_data['Sale_price'] = all_data['Sale_price'].astype(str).str.replace(',', '').str.replace('$', '').astype(float)

# Resampling the data to get monthly averages
all_data.set_index('Sale_date', inplace=True)
resampled_data = all_data['Sale_price'].resample('M').mean()

# Fitting the SARIMA model on the resampled data (ignoring NaN values)
sarima_model = SARIMAX(resampled_data.dropna(), order=(5, 1, 0), seasonal_order=(1, 1, 1, 12))
sarima_fit = sarima_model.fit()

# Forecasting the next 12 months
sarima_forecast = sarima_fit.get_forecast(steps=12).predicted_mean

# Plotting the original data and the SARIMA forecast
plt.figure(figsize=(15, 6))
plt.plot(resampled_data.index, resampled_data.values, label='Original')
plt.plot(pd.date_range(resampled_data.index[-1], periods=12, freq='M'), sarima_forecast, label='SARIMA Forecast')
plt.title('SARIMA Forecast of Property Sale Prices')
plt.xlabel('Date')
plt.ylabel('Sale Price')
plt.legend()
plt.show()


/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.01915D+01    |proj g|=  2.26341D+00



At iterate    5    f=  9.32329D+00    |proj g|=  7.23107D-02



At iterate   10    f=  9.30579D+00    |proj g|=  4.87975D-03



At iterate   15    f=  9.30553D+00    |proj g|=  7.93129D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8     16     21      1     0     0   1.246D-04   9.306D+00
  F =   9.3055259187205337     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [ ]:
# Importing necessary libraries for data manipulation and time series analysis
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# Reading the data from the CSV files
data_2008 = pd.read_csv('2008-property-sales-data.csv')
data_2018 = pd.read_csv('2018-property-sales-data.csv')
data_2022 = pd.read_csv('armslengthsales_2022_valid_20230404.csv')

# Concatenating the data into a single DataFrame
all_data = pd.concat([data_2008, data_2018, data_2022])

# Converting 'Sale_date' to datetime format and 'Sale_price' to float
all_data['Sale_date'] = pd.to_datetime(all_data['Sale_date'])

# Convert 'Sale_price' to a string, remove ',' and '$', and then convert it back to float
all_data['Sale_price'] = all_data['Sale_price'].astype(str).str.replace(',', '').str.replace('$', '').astype(float)

# Resampling the data to get monthly averages
all_data.set_index('Sale_date', inplace=True)
resampled_data = all_data['Sale_price'].resample('M').mean()

# Displaying the first few rows of the resampled data
resampled_data.head()


In [ ]:
# Importing necessary libraries for data manipulation and time series analysis
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# Reading the data from the CSV files
data_2008 = pd.read_csv('2008-property-sales-data.csv')
data_2018 = pd.read_csv('2018-property-sales-data.csv')
data_2022 = pd.read_csv('armslengthsales_2022_valid_20230404.csv')

# Concatenating the data into a single DataFrame
all_data = pd.concat([data_2008, data_2018, data_2022])

# Converting 'Sale_date' to datetime format
all_data['Sale_date'] = pd.to_datetime(all_data['Sale_date'])

# Resampling the data to get monthly averages
all_data.set_index('Sale_date', inplace=True)
resampled_data = all_data['Sale_price'].resample('M').mean()

# Displaying the first few rows of the resampled data
resampled_data.head()

In [ ]:
# Importing SARIMA model from statsmodels
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Fitting the SARIMA model on the resampled data (ignoring NaN values)
sarima_model = SARIMAX(resampled_data.dropna(), order=(5, 1, 0), seasonal_order=(1, 1, 1, 12))
sarima_fit = sarima_model.fit()

# Forecasting the next 12 months
sarima_forecast = sarima_fit.get_forecast(steps=12).predicted_mean

# Plotting the original data and the SARIMA forecast
plt.figure(figsize=(15, 6))
plt.plot(resampled_data.index, resampled_data.values, label='Original')
plt.plot(pd.date_range(resampled_data.index[-1], periods=12, freq='M'), sarima_forecast, label='SARIMA Forecast')
plt.title('SARIMA Forecast of Property Sale Prices')
plt.xlabel('Date')
plt.ylabel('Sale Price')
plt.legend()
plt.show()

/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.01915D+01    |proj g|=  2.26341D+00



At iterate    5    f=  9.32329D+00    |proj g|=  7.23107D-02



At iterate   10    f=  9.30579D+00    |proj g|=  4.87975D-03



At iterate   15    f=  9.30553D+00    |proj g|=  7.93129D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8     16     21      1     0     0   1.246D-04   9.306D+00
  F =   9.3055259187205337     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


## Conclusions

In this project, we aimed to analyze real estate sales data to identify seasonal patterns, long-term trends, and to forecast future trends. Here are the key takeaways:

1. **Data Cleaning and Preprocessing**: The data from multiple years was successfully cleaned and merged. We handled missing values and converted data types where necessary.

2. **Exploratory Data Analysis (EDA)**: We conducted an initial EDA to understand the distribution of sale prices, the volume of sales over time, and other key metrics. This helped us identify seasonality and trends in the data.

3. **Time Series Decomposition**: We decomposed the time series data to isolate the seasonal and trend components, which provided valuable insights into the cyclical nature of property sales.

4. **SARIMA Model**: We employed a Seasonal ARIMA model to forecast property sale prices for the next 12 months. The model captured the seasonality effectively and provided a reasonable forecast.

5. **Forecast Insights**: The forecast suggests a continuation of the current trends and seasonality. This information can be valuable for various stakeholders including investors, real estate agents, and policy makers.

6. **Future Work**: Further analysis could include testing more advanced models, conducting anomaly detection, and performing a more granular analysis by property type or location.

This project demonstrates a structured approach to time series analysis and provides a foundation for more complex analyses in the future.